# Optimizing Financial Aid Resources

**Notebook by [Dev Nambi](https://blogs.uw.edu/fractals/)**


### Goals

This notebook uses the enrollment prediction model created elsewhere and optimizes the distribution of funds given the constraints involved (total scholarship funding, explainability).


### Process

* A. Define constraints, such as the average SAT score, or average high school GPA, the different scholarship values, and the number of students to admit.
* B. Load in student data, software libraries, and model objects.
* C. Pass 1 - Arbitrary Predictions
* D. Calculate and visualize yield, net revenue, and scholarship info per segment.
* E. Pass 2 - Transfer Learning
* F. Calculate and visualize yield, net revenue, and scholarship info in an explainable way.
* G. Save off the results for later use.


### Resources

* https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/pareto-optimized-road-trip/optimized-state-capitols-trip.ipynb

## Part A - Define Goals and Constraints

Next we're going to have to specify constraints we care about:

* **Maximum Student Enrollment** - The maximum number of domestic non-resident students you would like to enroll
* **Average Student SAT** - The minimum average SAT for domestic non-resident students you're willing to accept
* **Award Amounts** - The different scholarship amounts to award.
* **Scholarship Fund Amount** - The amount of money to distribute via Purple and Gold scholarships

#### First, Define Constraints

In [1]:
fund_amount = 3000000   #3 million dollars

maximum_student_enrollment = 2100

average_student_sat = 1090

tuition_amount = 34143

#scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
scholarship_amounts = [0,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
#scholarship_amounts = [0,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000]
max_scholarship = max(scholarship_amounts)

number_of_buckets = 20

cutoff_threshold = 0.5

## Part B - Load in student data, software libraries, and model objects

### Part B1 - Load software libraries

In [2]:
# Load in software libraries
import os
import sys
import pandas as pd
import numpy as np
import pickle
import random
import copy
from tqdm import tqdm

# Get data-munging libraries
from sklearn import preprocessing
from sklearn import decomposition

# Get ML libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier

# Get Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Genetic algorithm
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


### Part B2 - Load  objects

In [4]:
model_location = 'boosting_model.pkl'

with open(model_location, 'rb') as model_input:
    predict_model = pickle.load(model_input)

#print (str(predict_model))

df = pd.read_csv('Transformed_Data_set_v3.csv')

propensity_df = pd.read_csv('Propensity_and_Tuition.csv')

In [5]:
propensity_df.head()

MinScore  MaxScore  TuitionEffect
0     0.000     0.068        0.00644
1     0.068     0.083        0.00600
2     0.083     0.095        0.00600
3     0.095     0.100        0.00600
4     0.100     0.104        0.00600

### Part C - First Pass, Arbitrary Predictions

1. Get the data into a handy format
2. Add propensity scores
3. Predict students who will come with no PuGo
4. For each scholarship amount, predict students who will switch and attend

**Reminder Constraints**

fund_amount
average_student_sat
award_amounts
maximum_student_enrollment

*Coding Notes*

* df_2 is used for predictions (remove IsEnrolledOrMatriculated)
* df_3 has the results we use, the scholarship amounts

In [6]:
df_IsEnrolled = df.IsEnrolledOrMatriculated
df_2 = df.copy(deep=True)
df_2.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)

In [7]:
df_3 = df.copy(deep=True)
#df_3 = pd.DataFrame(model_scalar.inverse_transform(df_2), columns=df_2.columns)
#df_3['IsEnrolledOrMatriculated'] = df_IsEnrolled
#df_3.describe()

In [8]:
df_3['PuGo_Amount'] = 0
df_3['EnrollPrediction'] = 0
df_3['EnrollPredictionTemp'] = 0
df_3['OriginalTuition_After_Grants'] = df_3['tuition_after_grants']

In [9]:
df_3.tuition_after_grants.describe()

count    15736.000000
mean     32172.000254
std       3262.487904
min      19476.000000
25%      28291.000000
50%      34791.000000
75%      34791.000000
max      34791.000000
Name: tuition_after_grants, dtype: float64

In [10]:
df_3.columns

Index([u'IsEnrolledOrMatriculated', u'EFC', u'Fisap_Income', u'WorkLoans',
       u'TuitionRemaining', u'TotalBudgetRemaining', u'HighSchoolGPA',
       u'TestScore', u'AcademicScore', u'HolisticScore1', u'HolisticScore2',
       u'UWAttraction_TotalCount', u'UWAttraction_TotalPct',
       u'UWAttraction_ProdPct', u'UWLocationAttraction_TotalCount',
       u'UWLocationAttraction_TotalPct', u'UWEdGoals_TotalCount',
       u'UWEdGoals_TotalPct', u'UWEdGoals_ProdPct', u'LowerIncomePct',
       u'LowerIncomeCount', u'HigherIncomeCount', u'LowerParentEdPct',
       u'LowerParentEdCount', u'HigherParentEdCount',
       u'CollegeLocationPct_BorderState', u'FamilyIncomePct_140to200K',
       u'FirstGenerationPct_Not1stGen', u'ParentEducationPct_NoResponse',
       u'BioAndMed', u'CompSci', u'Eng', u'Business', u'PsychAnthro',
       u'Nursing', u'Com', u'English', u'IntlStudies', u'Math', u'PhysicsAstr',
       u'Environ', u'Law', u'ArtDesign', u'DeltaParentEdMinusAcademic',
       u'DeltaAcad

In [11]:
def get_tuition_effect(prediction_score):
    tuition_score = propensity_df.TuitionEffect[(prediction_score >= propensity_df.MinScore) & (prediction_score < propensity_df.MaxScore)].head(1)
    return tuition_score

In [12]:
# test the function
get_tuition_effect(0.714596)

22    0.01939
Name: TuitionEffect, dtype: float64

### Part C1 - Predict which students will attend with no PuGo scholarship

In [13]:
# df_2.ix[:,0:31] <- use this to ignore columns added at the end
df_3['PredictionScore'] = predict_model.predict_proba(df_2)[:, 1]
df_3['EnrollPredictionTemp'] = predict_model.predict(df_2)

#df_3.describe()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [184]:
df_3['TuitionEffect'] = df_3.apply(lambda row: float(get_tuition_effect(row['PredictionScore'])), axis=1)
df_3.loc[df_3.TuitionEffect < 0, 'TuitionEffect'] = 0

In [185]:
df_3.TuitionEffect.describe()

count    18759.000000
mean         0.006652
std          0.002163
min          0.006000
25%          0.006000
50%          0.006440
75%          0.006440
max          0.019390
Name: TuitionEffect, dtype: float64

In [186]:
df_3['EnrollPrediction'] = df_3['EnrollPredictionTemp']

In [187]:
print ('Students who will enroll with no PuGo funding:',df_3.EnrollPrediction.sum())

('Students who will enroll with no PuGo funding:', 923L)


## Part D - Calculate and visualize yield, net revenue, and scholarship info per segment.

In [188]:
predicted_pugo_offered = df_3.PuGo_Amount.sum()
print ('PuGo Offered',predicted_pugo_offered)

predicted_pugo_use = df_3.PuGo_Amount[df_3['EnrollPrediction'] == 1].sum()
print ('PuGo Use',predicted_pugo_use)

('PuGo Offered', 0L)
('PuGo Use', 0L)


In [189]:
predicted_student_count = df_3.EnrollPrediction.sum()
total_student_count = df_3.EnrollPrediction.count()

print ('Predicted students:',predicted_student_count)
print ('Admitted students:',total_student_count)
print ('Predicted yield:',predicted_student_count * 100.0 / total_student_count)

('Predicted students:', 923L)
('Admitted students:', 18759)
('Predicted yield:', 4.92030492030492)


In [190]:
predicted_net_revenue = df_3.tuition_after_grants[df_3['EnrollPrediction'] == 1].sum()
print ('Predicted Net Revenue:',predicted_net_revenue)

('Predicted Net Revenue:', 30212093L)


In [191]:
predicted_avg_sat = df_3.TestScore[df_3['EnrollPrediction'] == 1].mean()
print ('Predicted Average SAT:',predicted_avg_sat)

('Predicted Average SAT:', 1296.679306608884)


## Part E - Pass 2, Transfer Learning and Genetic Optimization

* Useful resources, https://github.com/DEAP/notebooks/blob/master/OneMax.ipynb
* https://github.com/DEAP/notebooks/blob/master/SIGEvolution.ipynb
* http://deap.readthedocs.io/en/master/tutorials/basic/part1.html

### E1 - Assign students to 'buckets' based on academic performance

* Assign each student to a bucket
* Analyze and report on the characteristics of each bucket

In [192]:
df_3['BucketRank'] = df_3['AcademicScore'].rank(ascending=1)

bucket_count = 20
rows_per_bucket = df_3.BucketRank.count() / bucket_count

df_3['Bucket'] = np.floor(df_3['BucketRank'] / rows_per_bucket)

In [193]:
for i in range(0,20):
    high_school_gpa = df_3.HighSchoolGPA[df_3['Bucket'] == i].sum()
    min_high_school_gpa = df_3.HighSchoolGPA[df_3['Bucket'] == i].quantile(0.2)
    max_high_school_gpa = df_3.HighSchoolGPA[df_3['Bucket'] == i].quantile(0.8)
    test_score = df_3.TestScore[df_3['Bucket'] == i].sum()
    min_test_score = df_3.TestScore[df_3['Bucket'] == i].quantile(0.2)
    max_test_score = df_3.TestScore[df_3['Bucket'] == i].quantile(0.8)
    academic_score = df_3.AcademicScore[df_3['Bucket'] == i].sum()
    holistic_1 = df_3.HolisticScore1[df_3['Bucket'] == i].quantile(0.99)
    holistic_2 = df_3.HolisticScore1[df_3['Bucket'] == i].quantile(0.99)
    efc = df_3.EFC[df_3['Bucket'] == i].sum()
    students = df_3.HighSchoolGPA[df_3['Bucket'] == i].count()
    print ('Bucket',i,students, high_school_gpa*1.0/students, min_high_school_gpa, max_high_school_gpa, test_score*1.0/students, min_test_score, max_test_score, academic_score*1.0 / students, efc*1.0/students, holistic_1, holistic_2)



#df_3[df_3['Bucket'] == i].columns

('Bucket', 0, 937, 2.8069903948772654, 2.7200000000000006, 3.1099999999999994, 997.9188900747065, 910.0, 1240.0, 0.6900403548559233, 28012.03415154749, 0.0, 0.0)
('Bucket', 1, 937, 3.2911099252934912, 3.19, 3.35, 1090.6083244397012, 1012.0000000000001, 1305.0, 0.8016053895410887, 32442.321237993598, 0.0, 0.0)
('Bucket', 2, 937, 3.413351120597654, 3.33, 3.46, 1184.0448239060831, 1090.0, 1340.0, 0.8363413153681967, 38946.38954108858, 0.0, 0.0)
('Bucket', 3, 937, 3.5068409818569872, 3.44, 3.56, 1232.6734258271078, 1130.0, 1375.0, 0.8607668423159028, 43218.47491995731, 0.0, 0.0)
('Bucket', 4, 936, 3.5740491452991443, 3.52, 3.63, 1275.048076923077, 1150.0, 1380.0, 0.8790212005876058, 42095.77457264957, 0.0, 0.0)
('Bucket', 5, 937, 3.6271077908217673, 3.57, 3.68, 1302.187833511206, 1185.0, 1415.0, 0.892840514941302, 44263.47918890075, 0.0, 0.0)
('Bucket', 6, 936, 3.672478632478631, 3.62, 3.72, 1318.6805555555557, 1220.0, 1430.0, 0.9040280114850426, 38062.72222222222, 0.0, 0.0)
('Bucket', 7, 

**Make a single function that evaluates a scholarship 'bucket list' and returns relevant metrics**

Note: this is the most important bit of code in this entire notebook.

The function returns:

   * net tuition revenue
   * scholarship amount
   * students enrolled
   * median test score
   * yield 

In [232]:
def parse_scholarships_base(buckets, include_bucket_counts):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    tuition_revenue = 0
    scholarship_used = 0
    students = 0
    total_test_score = 0
    
    buckets = list(buckets)
    
    for i in range(len(buckets)):
        bucket_amount = buckets[i]
        #df_3.PuGo_Amount[df_3['Bucket'] == i] = bucket_amount
        #df_3.PuGo_Amount[] = bucket_amount
        df_3.loc[df_3.Bucket == i, 'PuGo_Amount'] = bucket_amount
    #
    #
    #
    #df_3['tuition_after_grants'] = df_3['OriginalTuition_After_Grants'] - df_3['PuGo_Amount']
    df_3['PredictionScoreTemp'] = df_3['PredictionScore'] + df_3['TuitionEffect']*df_3['PuGo_Amount']
    #
    df_3.loc[df_3.PredictionScoreTemp >= cutoff_threshold, 'EnrollPredictionTemp'] = 1
    df_3.loc[df_3.PredictionScoreTemp < cutoff_threshold, 'EnrollPredictionTemp'] = 0
    #
    students = df_3.EnrollPredictionTemp[df_3['EnrollPredictionTemp'] == 1].count()
    student_grads = df_3['5yr_grad_rate'][df_3['EnrollPredictionTemp'] == 1].sum()
    scholarship_used = df_3.PuGo_Amount[df_3['EnrollPredictionTemp'] == 1].sum()
    #tuition_revenue =  students * tuition_amount
    tuition_revenue =  (tuition_amount * df_3['5yr_grad_rate'] * df_3['EnrollPredictionTemp']).sum()
    total_test_score = df_3.TestScore[df_3['EnrollPredictionTemp'] == 1].sum()
    total_students = df_3.EnrollPredictionTemp.count()
    
    if include_bucket_counts:
        # look at the information per bucket
        for i in range(len(buckets)):
            bucket_students = df_3.EnrollPredictionTemp[(df_3['EnrollPredictionTemp'] == 1) & (df_3['Bucket'] == i)].count()
            #bucket_student_grads = df_3['5yr_grad_rate'][df_3['EnrollPredictionTemp'] == 1 & (df_3['Bucket'] == i)].sum()
            bucket_scholarship_used = df_3.PuGo_Amount[(df_3['EnrollPredictionTemp'] == 1) & (df_3['Bucket'] == i)].sum()
            bucket_revenue = bucket_students * tuition_amount
            bucket_test_score = df_3.TestScore[(df_3['EnrollPredictionTemp'] == 1) & (df_3['Bucket'] == i)].sum()
            bucket_total_students = df_3.EnrollPredictionTemp[df_3['Bucket'] == i].count()
            print ('Bucket ',i, bucket_students, bucket_scholarship_used, bucket_revenue - bucket_scholarship_used, bucket_test_score * 1.0 / bucket_students * 1.0, bucket_students * 1.0 / bucket_total_students)
        
    # return everything
    return tuition_revenue - scholarship_used, scholarship_used, students, total_test_score * 1.0 / students * 1.0, students * 1.0 / total_students

In [233]:
def parse_scholarships(buckets):
    """
        This function returns the net tuition revenue, scholarship used, 
        student count, and average test score
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships_base(buckets, False)
    return net_revenue, scholarship_used, students, avg_test_score, student_yield

In [234]:
buckets = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(buckets)

print ('Net Revenue is ', net_revenue)
print ('Scholarship Used Is', scholarship_used)
print ('Student Count Is ',students)
print ('Average Test Score Is', avg_test_score)
print ('Student Yield Is',student_yield)

('Net Revenue is ', 69022891.97121698)
('Scholarship Used Is', 0L)
('Student Count Is ', 2602)
('Average Test Score Is', 1316.370099923136)
('Student Yield Is', 0.13870675409136948)


### Create the Evaluation Function

In [249]:
def eval_scholarships(bucket_combo):
    """
        This function returns the net tuition revenue as well as the number of students
        predicted to enroll.
        
        The genetic algorithm will favor scholarship distributinon that
        bring more money or more students.
    """
    net_revenue, scholarship_used, students, avg_test_score, student_yield = parse_scholarships(bucket_combo)
    
    net_revenue_to_return = net_revenue
    if scholarship_used > fund_amount:
        net_revenue_to_return = net_revenue_to_return * 1.0 * fund_amount / (scholarship_used*1.4)
    elif students > maximum_student_enrollment:
        net_revenue_to_return = net_revenue_to_return * 1.0 * maximum_student_enrollment / (students*1.4) 
    
    
    return net_revenue_to_return, avg_test_score

### Create the mutation function

In [250]:
np.random.choice([amt for amt in scholarship_amounts if amt > 10000],1)[0]

11000

In [251]:
def compare_lists(x, y):
    return 1 if len(frozenset(x).intersection(y)) == max(len(x),len(y)) else 0

In [252]:
a = [1,2,3,4,5]
b = [1,2,3,4,5]

compare_lists(a,b)

1

In [253]:
scholarship_amounts_2 = list(scholarship_amounts)

print (id(scholarship_amounts))
print (id(scholarship_amounts_2))
compare_lists(scholarship_amounts, scholarship_amounts_2)

4656443904
4663328712


1

In [254]:
def mutate_scholarships_base(bucket_combo):
    original_combo = list(bucket_combo)

**Make a single function that takes a scholarship 'bucket list' and *changes* it to something very similar but not exactly the same**

Note: this is the second-most important bit of code in this entire notebook, after the evaluation function

In [255]:
def mutation_operator(bucket_combo):
    """
        This function applies a random change to one set of scholarships
        
            - Init: create a bucket with 20 '0' values if it's not the right size
            - Increase: Add a scholarship to a random bucket
            - Reduce: Reduce a scholarship in a random bucket
            - Swap: Swaps the scholarship amounts of two buckets
    """
    possible_mutations = ['swap']
    original_combo = list(bucket_combo)
    bucket_list = list(bucket_combo)
    itercount = 0
    #print ('Bucketlist',bucket_list)
    #print ('Originalcombo',original_combo)
    #print ('Comparelists',compare_lists(original_combo, bucket_list))
    
    while compare_lists(original_combo, bucket_list) > 0: #keep going if it's not the same
        itercount = itercount + 1
        print ('Itercount',itercount)
        if np.min(bucket_combo) < max_scholarship:
            possible_mutations.append('increase')
        if np.max(bucket_combo) > 0:
            possible_mutations.append('reduce')
        #
        mutation_type = random.sample(possible_mutations, 1)[0]
        #print 'Mutation type: ',mutation_type
        #
        # Increase mutation
        if mutation_type == 'increase':
            to_increase = True
            while to_increase:
                index_to_increase = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index_to_increase] < (max_scholarship):
                    bucket_combo[index_to_increase] = np.random.choice([amt for amt in scholarship_amounts if amt > bucket_combo[index_to_increase]],1)[0]
                    to_increase = False
        #
        # Reduce mutation
        elif mutation_type == 'reduce':
            index_to_delete = random.randint(0, number_of_buckets - 1)
            if bucket_combo[index_to_delete] > 0:
                bucket_combo[index_to_delete] = np.random.choice([amt for amt in scholarship_amounts if amt < bucket_combo[index_to_delete]],1)[0]
        #
        # Swap mutation
        elif mutation_type == 'swap':
            index1 = random.randint(0, number_of_buckets - 1)
            index2 = index1
            while index2 == index1:
                index2 = random.randint(0, number_of_buckets - 1)
                if bucket_combo[index1] == bucket_combo[index2]:
                    index2 = index1
            #
            bucket_combo[index1], bucket_combo[index2] = bucket_combo[index2], bucket_combo[index1]
        #
        bucket_combo.sort()
        bucket_list = list(bucket_combo)
    # now that things are sorted, return the result
    return bucket_combo,

### Create the initial attribute function

In [256]:
def random_scholarshipamount():
    return np.random.choice(scholarship_amounts, 1)[0]

In [257]:
def random_individual():
    random_buckets = np.random.choice(scholarship_amounts, size=number_of_buckets, replace=True).tolist()
    random_buckets.sort()
    return creator.Individual(random_buckets)

#### Type Creation

First step with DEAP is to create the required types. Usually the types created are the fitness and the individual. For the OneMax problem, we want to have a solution with as many ones as possible. Thus we need a maximizing fitness and a individual that is a sorted container, we'll choose the standard list.

Type creation is done by calling the function create in the creator module. This function takes two mandatory arguments and additional optional arguments. The first argument is the actual name of the type that we want to create. The second argument is the base classe that the new type created should inherit from. Finally, the optional arguments are members to add to the new type.

In [258]:
creator.create('FitnessMulti', base.Fitness, weights=(1.0, 0.001))
creator.create('Individual', list, fitness=creator.FitnessMulti)

In [259]:
all_buckets = range(0, bucket_count) #there are 20 buckets to start with
bucket_combinations = {}

buckets = [0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [260]:
ind = creator.Individual([1, 0, 1, 1, 0])

print(ind)
print(type(ind))
print(type(ind.fitness))
print(type(all_buckets))

[1, 0, 1, 1, 0]
<class 'deap.creator.Individual'>
<class 'deap.creator.FitnessMulti'>
<type 'list'>


#### Toolbox

The toolbox is intended to store functions with their arguments under standard aliases for uses in algorithms. Functions are registered by a function call with two mandatory arguments, the alias to give to the function and the function it will be associate with. Any additional argument will be given as argument when the alias is called.

In [261]:
toolbox = base.Toolbox()

This first one, attr_bool, calls randint from the random module with arguments (0, 1) to create an integer in the interval $[0, 1]$.

In [262]:
toolbox.register("attr_scholarshipamount", random_scholarshipamount)

In [263]:
bit = toolbox.attr_scholarshipamount()
print("bit is of type %s and has value\n%s" % (type(bit), bit))

bit is of type <type 'numpy.int64'> and has value
5000


The second function, individual, when called, will use the initRepeat function made available in the tools module to fill an Individual class with what is produced by 10 calls to the previously defined attr_scholarship function. 

* Note: this has been deprecated in favor of a custom function, random_individual, which returns sorted results

In [264]:
toolbox.register("individual", random_individual)

In [265]:
ind = toolbox.individual()
print("ind is of type %s and contains %d bits\n%s" % (type(ind), len(ind), ind))

ind is of type <class 'deap.creator.Individual'> and contains 20 bits
[0, 0, 5000, 5000, 5000, 5000, 6000, 7000, 8000, 8000, 9000, 9000, 10000, 11000, 12000, 12000, 13000, 13000, 13000, 15000]


The same thing is done for the population function.

In [266]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [267]:
pop = toolbox.population(n=3)
print("pop is of type %s and contains %d individuals\n%s" % (type(pop), len(pop), pop))

pop is of type <type 'list'> and contains 3 individuals
[[0, 0, 0, 4000, 4000, 5000, 6000, 6000, 9000, 10000, 10000, 11000, 11000, 12000, 13000, 13000, 14000, 14000, 14000, 15000], [0, 0, 0, 6000, 6000, 7000, 8000, 8000, 9000, 9000, 10000, 11000, 11000, 11000, 11000, 11000, 12000, 12000, 13000, 13000], [0, 4000, 4000, 6000, 6000, 6000, 7000, 7000, 7000, 8000, 9000, 9000, 9000, 9000, 10000, 10000, 10000, 13000, 14000, 14000]]


## Genetic Operators

Registering the operators and their default arguments in the toolbox is done as follows:

The evaluation is given the alias evaluate. Having a single argument being the individual to evaluate we don’t need to fix any, the individual will be given later in the algorithm. 

In [268]:
toolbox.register('evaluate', eval_scholarships)

In [269]:
ind = toolbox.individual()
evaluation = toolbox.evaluate(ind)
print("Eval is of type %s and contains \n%s" % (type(evaluation), evaluation))

Eval is of type <type 'tuple'> and contains 
(3461715.5287676337, 1316.7580894503972)


The two points crossover function is registered the same way under the alias mate. 

In [270]:
toolbox.register("mate", tools.cxTwoPoint)

The mutation, for its part, needs an argument to be fixed (the independent probability of each attribute to be mutated indpb). In the algorithms the mutate() function is called with the signature toolbox.mutate(mutant). This is the most convenient way because each mutation takes a different number of arguments, having those arguments fixed in the toolbox leave open most of the possibilities to change the mutation (crossover, selection, or evaluation) operator later in your researches.

In [271]:
toolbox.register('mutate', mutation_operator)

We can for example mutate an individual and expect 10% of its attributes to be flipped.

In [272]:
ind = toolbox.individual()
print(ind)
for i in range(0,2):
    toolbox.mutate(ind)
    revenue = toolbox.evaluate(ind)
    print('Revenue of ',revenue,'comes from',ind)

[0, 5000, 6000, 6000, 7000, 7000, 7000, 7000, 9000, 9000, 10000, 10000, 11000, 11000, 11000, 12000, 13000, 15000, 15000, 15000]
('Revenue of ', (3701532.782566853, 1332.2742783620497), 'comes from', [0, 5000, 6000, 6000, 7000, 7000, 7000, 7000, 9000, 9000, 10000, 10000, 11000, 11000, 11000, 12000, 13000, 15000, 15000, 15000])
('Revenue of ', (3701532.782566853, 1332.2742783620497), 'comes from', [0, 5000, 6000, 6000, 7000, 7000, 7000, 7000, 9000, 9000, 10000, 10000, 11000, 11000, 11000, 12000, 13000, 15000, 15000, 15000])


Finally, the selection operator is registered under the name select and the size of the tournament set to 3.

* https://pdfs.semanticscholar.org/2fb8/850864d71d6b9b30a6f4729a82cba456b027.pdf
* http://deap.readthedocs.io/en/master/api/tools.html

In [273]:
toolbox.register("select", tools.selTournament, tournsize=5)

### Evolving the Population

The main program shall be defined in a main function. All previous declarations are put in the global scope of the module to make created objects available for further use in other user experiments.

The main program is very simple. It consists of generating a population, and giving it to the algorithm for it to evolve a solution. Here we will employ some helpful introspection tools such as Statistics and a Hall of Fame. The statistics are computed using numpy functions on the population, and the hall of fame keeps track of the best individuals that ever appeared during the evolution. The algorithm take as arguments, among other, the population and the toolbox. Once the evolution is finished the population contains the individuals from the last generation.

In [274]:
pop = toolbox.population(n=1000)

In [275]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)

In [276]:
fund_amount = 3000000   #3 million dollars

* http://deap.readthedocs.io/en/master/api/algo.html

In [ ]:
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.25, mutpb=0.8, ngen=40, stats=stats, halloffame=hof, verbose=True)

#cxpb 0.5 or 0? - The probability of mating two individuals.
#mutpb 0.2 or 1.0? - The probability of mutating an individual.

In [ ]:
print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))

#print ('Evaluation of best individual:', parse_scholarships(hof[0]))
print ('Bucket, Students, ScholarshipUsed, NetRevenue, AvgTestScore, Yield')
print ('Evaluation of best individual detailed:', parse_scholarships_base((hof[0]), True))

import matplotlib.pyplot as plt
gen, avg, max_ = logbook.select("gen", "avg", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, max_, label="maximum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

In [285]:
bucket_list = [1000, 1000, 1000, 1000, 1000, 1000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 3500, 4500, 5500, 8000, 8000, 9000]
bucket_list = [0, 0, 0, 0, 0, 0, 0, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 4000, 5000, 5000, 6000, 6000, 9000]
bucket_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4000, 4000, 4000, 4000, 6000, 6000, 8000, 8000, 9000]
bucket_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6000, 7000, 8000, 9000, 11000]
parse_scholarships_base(bucket_list, True)

('Bucket ', 0, 42, 0L, 1434006, 877.9761904761905, 0.044823906083244394)
('Bucket ', 1, 32, 0L, 1092576, 1117.96875, 0.034151547491995733)
('Bucket ', 2, 40, 0L, 1365720, 1245.375, 0.042689434364994665)
('Bucket ', 3, 65, 0L, 2219295, 1248.6923076923076, 0.069370330843116335)
('Bucket ', 4, 64, 0L, 2185152, 1280.0, 0.068376068376068383)
('Bucket ', 5, 89, 0L, 3038727, 1313.7078651685392, 0.094983991462113129)
('Bucket ', 6, 65, 0L, 2219295, 1260.6153846153845, 0.069444444444444448)
('Bucket ', 7, 48, 0L, 1638864, 1262.9166666666667, 0.051118210862619806)
('Bucket ', 8, 47, 0L, 1604721, 1299.2553191489362, 0.050160085378868728)
('Bucket ', 9, 41, 0L, 1399863, 1346.7073170731708, 0.043944265809217578)
('Bucket ', 10, 47, 0L, 1604721, 1307.872340425532, 0.049893842887473464)
('Bucket ', 11, 31, 0L, 1058433, 1339.1935483870968, 0.033084311632870865)
('Bucket ', 12, 49, 0L, 1673007, 1347.4489795918366, 0.052406417112299465)
('Bucket ', 13, 42, 0L, 1434006, 1329.8809523809523, 0.044871794871

(108086856.91229314, 38490000L, 5441, 1416.8195184708693, 0.29004744389359777)